In [35]:
import random
import time

In [36]:
small_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]

In [37]:
def is_probable_prime(n, k=10):
    """Return True if n is probably prime (error ≤ 4⁻ᵏ), else False."""
    if n < 2:
        return False

    # Quick screening by small primes
    for p in small_primes:
        if n == p:
            return True
        if n % p == 0:
            return False

    # Write n-1 as 2^s * d with d odd
    d = n - 1
    s = (d & -d).bit_length() - 1
    d >>= s

    # k rounds of testing
    for _ in range(k):
        a = random.randrange(2, n - 1)
        # a = secrets.randbelow(n - 3) + 2  # for crypto
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            # no break → composite
            return False

    return True

In [ ]:
def run_tests(bit_size, trials=10000, k=5):
    false_positives = 0
    start = time.perf_counter()
    for _ in range(trials):
        # generate a random odd integer of the given bit size
        n = random.getrandbits(bit_size) | 1 | (1 << (bit_size-1))
        if is_probable_prime(n, k) and not all(n % p for p in small_primes):
            # worst-case: this flags a small prime as false positive
            false_positives += 1
    elapsed = time.perf_counter() - start
    avg_ms = (elapsed / trials) * 1000
    print(f"{bit_size}-bit: {trials} tests in {elapsed:.2f}s — "
      f"avg {avg_ms:.3f} ms/test, false positives: {false_positives}")

In [40]:
if __name__ == "__main__":
    for bit_size in range(3, 13):
        run_tests(pow(2, bit_size))
    